In [1]:
import os, re
import pandas as pd
import datetime
import nltk
import numpy as np

In [2]:
words_norust_nosw = pd.read_csv('../../Result/STM/alldf_norust_nonsw_t45_180223.csv')

In [3]:
words_norust_sw = pd.read_csv('../../Result/STM/alldf_norust_sw_t45_180223.csv')

In [4]:
words_rust_nosw = pd.read_csv('../../Result/STM/rust_nsw.csv')

In [5]:
words_rust_sw = pd.read_csv('../../Result/STM/rust_swing.csv')

In [6]:
rust_sw = list(words_rust_sw.loc[33])[1:]

In [28]:
#rust_sw

In [7]:
rust_nosw = list(words_rust_nosw.loc[12])[1:]

In [8]:
norust_sw = list(words_norust_sw.loc[35])[1:]

In [9]:
norust_nosw = list(words_norust_nosw.loc[4])[1:]

In [10]:
pt = nltk.stem.PorterStemmer()

In [11]:
#alldf = pd.read_csv('../../Data/alldf_180223.csv')
#cov = pd.read_csv('../../Data/alldf_covariates.csv')

In [12]:
# filter_wordsls = ['free trade agreeement','free trade' ,'liberalization', 'liberalize' 'labor', 'labor union',
# 'export', 'grow', 'growth', 'manufacturing', 'world','subsidy', 'protect', 'protection', 
# 'economy', 'china','import', 'competition', 'compete', 'rust belt','mexico', 
# 'cheap', 'unemploy', 'unemployment', 'adjustment', 'adjust' ,'wto', 'nafta', 'dumping', 'anti-dumping',
# 'trade','unfair']

In [13]:
rustbelts = ['New York', 'Pennsylvania', 'West Virginia', 'Ohio', 'Indiana', 'Michigan',
             'Illinois', 'Iowa', 'Wisconsin', 'Missouri']

In [14]:
rustbelts_ex = ['Pennsylvania', 'West Virginia', 'Ohio', 'Indiana', 'Michigan',
             'Illinois', 'Iowa', 'Wisconsin', 'Missouri']

In [15]:
dem = ['Barack Obama', 'Hillary Clinton', 'John Edwards', 'Bill Richardson',
      'Christopher Dodd', 'Joe Biden', 'Bernie Sanders',  "Martin O'Malley", 
      'Lincoln Chafee', 'Jim Webb']

rep = ['John McCain', 'Mike Huckabee', 'Mitt Romney', 'Rudy Giuliani',
      'Fred Thompson', 'Ron Paul', 'Newt Gingrich', 'Rick Santorum', 'Rick Perry',
      'Jon Huntsman', 'Michele Bachmann', 'Herman Cain', 'Tim Pawlenty',
       'Donald J. Trump', 'John Kasich', 'Ted Cruz', 'Marco Rubio', 'Ben Carson',
      'Jeb Bush', 'Chris Christie', 'Carly Fiorina', 'Rand Paul', 'George Pataki',
      'Lindsey Graham','Scott Walker', 'Bobby Jindal',]

In [16]:
stops = nltk.corpus.stopwords.words('english')

In [17]:
stemmed_stop = list(map(lambda x: re.sub('\W', '', x), stops))
punc = list(map(pt.stem , stemmed_stop))

In [29]:
rootDir = '../../Data/'
d = {}
remarkls = []
interviewls = []
questionls = []
otherls = []
speechls = []
addressls = []
r = 0
i = 0
q = 0
o = 0
s = 0
fdls = []
pe_dict = {2008:{}, 2012:{}, 2016:{}}
for dirName, subdirList, fileList in os.walk(rootDir) :
    print('Found directory: %s' % dirName)
    print(dirName)
    c = 0
    for fname in fileList:        
        if fname.endswith(".txt"):  
            filepath = os.path.join(dirName, fname)
            if dirName == '../../Data/PE2008':  
                c += 1
                year = 2008
                d = pe_dict[year]
                speaker = fname[8:-4]
            else:
                c += 1
                year = fname[8:12]
                d = pe_dict[int(year)]
                speaker = fname[12:-4]
            if speaker in dem:
                partyid = 'Dem'
            if speaker in rep:
                partyid = 'Rep'
            d[speaker]={}
            with open(filepath, 'r', -1, "ISO-8859-1") as input_file:
                data = input_file.read()
                splited_speeches = data.split('\n\n\n')
                #print(splited_speeches)
                statement_num = 0
                for i in range(len(splited_speeches)):
                    statement_num += 1
                    one_speech = re.sub(r'\((?!k\))(.*?)\)', '', splited_speeches[i], flags= re.IGNORECASE)
                    one_speech_fin = re.sub(r'\[(.*?)\]', '', one_speech) 
                    t = one_speech_fin.replace('\x80\x94','').replace('\x80','')\
                                      .replace('\x94','').split('\n')
                    text = [item for item in t if item != ''\
                            if not item.startswith('To view')\
                            if not item.startswith('Citation:')]
                    whole_text = " ".join(text[2:])    
                    #whole_text5 = re.sub(r'[^\w\s]', '',whole_text31.lower())                    
                    tokens = nltk.tokenize.word_tokenize(whole_text.lower())
                    stemmed_tokens = [pt.stem(w) for w in tokens]
                    #punc = [re.sub(r'[^\w]', '', word.lower()) for word in stops]
                    unigrams = [word for word in stemmed_tokens if word not in punc]                    
                    fd = nltk.FreqDist(unigrams)
                    if text != []:
                        title = text[0]
                        dateobj = datetime.datetime.strptime(text[1], '%B %d, %Y')
                        remarkls.append([speaker, title])
                        if 'remarks' in text[0].lower():
                            remarkls.append([speaker, title])
                            ttype = 'remarks'
                        elif 'speech' in text[0].lower():
                            speechls.append([speaker, title])
                            ttype = 'speech'
                        elif 'interview' in text[0].lower():
                            interviewls.append([speaker, title])
                            ttype = 'interview'
                        elif ('question' or 'q&a') in text[0].lower():
                            questionls.append([speaker, title])
                            ttype = 'question'
                        elif 'address' in text[0].lower():
                            addressls.append([speaker, title]) 
                            ttype = 'address'
                        else:
                            otherls.append([speaker, title])
                            ttype = 'others'
                                                
                        d[speaker][statement_num] = {'Author': speaker, 'Title': title,
                                                    'Year':year, 'Whole':splited_speeches[i],
                                                     'Clean': whole_text,
                                                     'Text': text,
                                                     'Date':dateobj,
                                                    'Type': ttype,
                                                     'PartyID':partyid}
                        
                        fdls.append(fd.most_common(10))
                        for word in rust_sw:
                            num = unigrams.count(word)
                        #num = len(re.findall(word, speech.lower()))
                            d[speaker][statement_num].update({word: num})
                    #print(statement_num)

Found directory: ../../Data/
../../Data/
Found directory: ../../Data/cb_2016_us_state_20m
../../Data/cb_2016_us_state_20m
Found directory: ../../Data/PE2008
../../Data/PE2008
Found directory: ../../Data/PE2012
../../Data/PE2012
Found directory: ../../Data/PE2016
../../Data/PE2016


In [37]:
#rust_nosw

In [38]:
#rust_sw

In [39]:
#norust_nosw

In [40]:
#norust_sw

In [9]:
#re.sub(r'\[(.*?)\]' , '','Thank you. (APPLAUS) (APPLAUSE AND CHEERS) [applause] Thank you. [applause] (k) (applause and cheers) [apple and bo] (cheers, applause.).')

In [54]:
swing  = pd.read_excel('../Data/swingstate_2008-2016.xlsx')

In [10]:
#re.findall(r'\((?!k\))\w+\)', '(k) i am, us dep economy (applause) (ph) (APPLAUSE)')

In [30]:
temp = {}
for year1, values1 in pe_dict.items():
    for author1, values2 in values1.items():
        for number, values3 in values2.items():
            #print(values3)
            temp.setdefault('Year1', []).append(year1)
            temp.setdefault('Author1', []).append(author1)
            temp.setdefault('No.', []).append(number)
            for key, value in values3.items():
                temp.setdefault(key, []).append(value)

In [31]:
df = pd.DataFrame(temp)

In [43]:
#df

In [44]:
df08 = df[df['Year1'] == 2008]
df12 = df[df['Year1'] == 2012]
df16 = df[df['Year1'] == 2016]

In [45]:
f08 = df08[(datetime.datetime(2005, 1, 1, 0, 0) <= df08['Date']) \
           & (df08['Date'] <= datetime.datetime(2008, 12, 31, 0, 0))]

In [46]:
f12 = df12[(datetime.datetime(2009, 1, 1, 0, 0) <= df12['Date']) \
           & (df12['Date'] <= datetime.datetime(2012, 12, 31, 0, 0))]

In [47]:
f16 = df16[(datetime.datetime(2013, 1, 1, 0, 0) <= df16['Date']) \
           & (df16['Date'] <= datetime.datetime(2016, 12, 31, 0, 0))]

In [48]:
refine = pd.concat([f08,f12,f16]).reset_index()

In [84]:
refine.columns

Index(['index', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID', 'Text',
       'Title', 'Type', 'Whole', 'Year', 'Year1', 'agreement', 'china', 'deal',
       'labor', 'manufactur', 'nafta', 'plant', 'trade', 'union', 'worker'],
      dtype='object')

In [85]:
refine_fin  = refine[['Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID', 'Text',
       'Title', 'Type', 'Whole', 'Year', 'Year1','agreement', 'china', 'deal',
       'labor', 'manufactur', 'nafta', 'plant', 'trade', 'union', 'worker']]

In [86]:
refine_fin.to_csv('../../Data/df_180226.csv')

In [21]:
#statedf = refine[['index', 'Author', 'No.', 'Title', 'Type', 'Year']]

In [22]:
#statedf.to_csv('df18023_addingstate.csv')

In [87]:
rdf = pd.read_csv('../../Data/df_180226.csv')

In [88]:
rdf.columns

Index(['Unnamed: 0', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'Text', 'Title', 'Type', 'Whole', 'Year', 'Year1', 'agreement', 'china',
       'deal', 'labor', 'manufactur', 'nafta', 'plant', 'trade', 'union',
       'worker'],
      dtype='object')

In [72]:
#pd.read_csv('../../Data/alldf_180223.csv').shape

In [89]:
covdf = pd.read_csv('../../Data/alldf_covariates2.csv')

In [90]:
covdf.columns

Index(['Unnamed: 0', 'Year', 'Year1', 'Author', 'Author1', 'Clean', 'Date',
       'No.', 'PartyID', 'State', 'Abb', 'Title', 'swing_last_08',
       'swing_last_12', 'swing_last_16', 'Rustbelt', 'unemployment',
       'challenger', 'governorp', 'primary'],
      dtype='object')

In [71]:
adddf = pd.read_excel('../Data/df18022_addingstate_afternoon.xlsx', sheet_name='df18022_addingstate')

In [73]:
adddf.columns

Index(['index', 'Author', 'No.', 'Title', 'Type', 'Year', 'State', 'Abb'], dtype='object')

In [133]:
merge_dtm = covdf.merge(rdf, left_on=['Author','No.','Year','Year1',
                                      'Date','PartyID', 'Title'],
                             right_on = ['Author','No.','Year','Year1','Date',
                                      'PartyID', 'Title'],
                            how='left')

In [134]:
merge_dtm.columns

Index(['Unnamed: 0_x', 'Year', 'Year1', 'Author', 'Author1_x', 'Clean_x',
       'Date', 'No.', 'PartyID', 'State', 'Abb', 'Title', 'swing_last_08',
       'swing_last_12', 'swing_last_16', 'Rustbelt', 'unemployment',
       'challenger', 'governorp', 'primary', 'Unnamed: 0_y', 'Author1_y',
       'Clean_y', 'Text', 'Type', 'Whole', 'agreement', 'china', 'deal',
       'labor', 'manufactur', 'nafta', 'plant', 'trade', 'union', 'worker'],
      dtype='object')

In [135]:
merge_dtm_fin = merge_dtm[['Unnamed: 0_x', 'Year', 'Year1', 
           'Author', 'Author1_x', 'Clean_x','Date', 'No.', 
           'PartyID', 'State', 'Abb', 'Title', 'swing_last_08',
           'swing_last_12', 'swing_last_16', 'Rustbelt', 
           'unemployment','challenger', 'governorp', 'primary', 
           'Text', 'Type', 'Whole','agreement', 'china', 'deal',
           'labor', 'manufactur', 'nafta', 'plant', 'trade', 
           'union', 'worker']]
merge_dtm_fin.rename(columns = {'Author1_x':'Author1','Clean_x':'Clean'}, inplace = True)

/Users/misun/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [126]:
merge_dtm.to_csv('../../Data/alldf_covariates_adddtm.csv')

In [130]:
dtm = pd.read_csv('../../Data/alldf_covariates_adddtm.csv')

In [131]:
dtm.columns

Index(['Unnamed: 0', 'Unnamed: 0_x', 'Year', 'Year1', 'Author', 'Author1',
       'Clean', 'Date', 'No.', 'PartyID', 'State', 'Abb', 'Title',
       'swing_last_08', 'swing_last_12', 'swing_last_16', 'Rustbelt',
       'unemployment', 'challenger', 'governorp', 'primary', 'Unnamed: 0_y',
       'Author1_y', 'Clean_y', 'Text', 'Type', 'Whole', 'agreement', 'china',
       'deal', 'labor', 'manufactur', 'nafta', 'plant', 'trade', 'union',
       'worker'],
      dtype='object')

In [74]:
merged = rdf.merge(adddf, left_on=['Author','No.','Year'],
                right_on = ['Author','No.','Year'], how='left')

In [75]:
merged.columns

Index(['Unnamed: 0', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'Text', 'Title_x', 'Type_x', 'Whole', 'Year', 'Year1', 'index',
       'Title_y', 'Type_y', 'State', 'Abb'],
      dtype='object')

In [78]:
swing.columns

Index(['year', 'state', 'swing_last2', 'swing_last', 'average'], dtype='object')

In [82]:
sw08 = swing[swing['year'] == 2008]
sw12 = swing[swing['year'] == 2012]
sw16 = swing[swing['year'] == 2016]

In [83]:
addingstatedf = merged[['Unnamed: 0', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
                        'State','Abb','Text', 'Title_x', 'Type_x', 'Whole', 'Year', 'Year1', 
                        'index','Title_y', 'Type_y']]

In [87]:
addstate08 = addingstatedf[addingstatedf['Year1'] == 2008]
addstate12 = addingstatedf[addingstatedf['Year1'] == 2012]
addstate16 = addingstatedf[addingstatedf['Year1'] == 2016]

In [107]:
alldf = pd.concat([addstate08,addstate12,addstate16])

In [108]:
statesw08 = alldf.merge(sw08, left_on=['Abb'],right_on = ['state'], how='left')
statesw08.rename(columns = {'swing_last':'swing_last_08'}, inplace = True)

In [110]:
statesw12 = statesw08.merge(sw12, left_on=['Abb'],right_on = ['state'], how='left')
statesw12.rename(columns = {'swing_last':'swing_last_12'}, inplace = True)

In [112]:
statesw16 = statesw12.merge(sw16, left_on=['Abb'],right_on = ['state'], how='left')
statesw16.rename(columns = {'swing_last':'swing_last_16'}, inplace = True)

In [120]:
alldf_sw = statesw16[['Year', 'Year1', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'State', 'Abb', 'Title_x', 'swing_last_08', 'swing_last_12', 'swing_last_16']]

In [123]:
alldf_sw ['Rustbelt'] = np.where(alldf_sw["State"].isin(rustbelts_ex), 1, 0)

/Users/misun/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [134]:
alldf_sw.rename(columns = {'Title_x':'Title'}, inplace = True)

/Users/misun/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [135]:
alldf_sw.columns

Index(['Year', 'Year1', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'State', 'Abb', 'Title', 'swing_last_08', 'swing_last_12',
       'swing_last_16', 'Rustbelt'],
      dtype='object')

In [141]:
alldf_sw.to_csv('../Data/alldf_180226.csv')

In [140]:
#pd.read_csv('alldf_180223.csv')

In [104]:
print(addstate08.shape)
print(addstate12.shape)
print(addstate16.shape)
print(statesw08.shape)
print(statesw12.shape)
print(statesw16.shape)

(685, 18)
(282, 18)
(280, 18)
(685, 23)
(282, 23)
(280, 23)


In [129]:
#df1["EU"] = np.where(df1["Country"].isin(EU), "EU", "Other")

In [109]:
#ny.to_csv('NY_sate_180223.csv', index = False)

In [41]:
#findf = addingstatedf[['Year', 'Year1','Author', 'Author1','Date', 'No.',
#!               'PartyID', 'State', 'Clean','Title_x', 'Whole']]

In [42]:
#findf.to_csv('../Data/df_final_180223.csv', index = False)

In [44]:
rust10 = findf[findf['State'].isin(rustbelts_ex)][['Author','Year1','No.','Date',
                                                'PartyID','State','Clean']]

In [45]:
rust10_not = findf[~findf['State'].isin(rustbelts_ex)][['Author','Year1','No.','Date',
                                                        'PartyID','State','Clean']]

In [46]:
rust10_not.groupby('Year1').count()

,Author,No.,Date,PartyID,State,Clean
Year1,,,,,,
2008,510,510,510,510,510,510
2012,178,178,178,178,178,177
2016,206,206,206,206,206,206


In [121]:
rust10_not.to_csv('non_rust_10_ex_180223.csv', index=False)

In [130]:
#pd.read_csv('non_rust_10_ex_180223.csv')

In [84]:
#rust10.to_csv('clean_rust_10_180223.csv')

In [85]:
#rust10_not.to_csv('clean_rust_10_not_180223.csv')

In [97]:
#clean = pd.read_csv('clean_rust_10_180223.csv')

In [99]:
#t = clean['Clean'].iloc[4]

In [118]:
#t

In [119]:
#clean[clean['Clean'].str.contains('[applause]')]

In [195]:
print(len(remarkls))
print(len(interviewls))
print(len(questionls))
print(len(otherls))
print(len(speechls))
print(len(addressls))

1069
178
2
17
53
79


In [42]:
#len(splited_speeches)

In [ ]:
temp = {}
for year1, values1 in pe_dict.items():
    for author1, values2 in values1.items():
        for number, values3 in values2.items():
            for num, values4 in values3.items():
                #print(values3)
                temp.setdefault('Doc.No.', []).append(number)
                temp.setdefault('Year1', []).append(year1)
                temp.setdefault('Author1', []).append(author1)
                temp.setdefault('No.', []).append(num)
                for key, value in values4.items():
                    temp.setdefault(key, []).append(value)